In [1]:
import os
import copy
import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict

import matplotlib.pyplot as plt

import pickle
import warnings

from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform
from scipy.stats import mannwhitneyu, spearmanr
from sklearn.base import clone
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import RocCurveDisplay, roc_curve, auc, precision_recall_curve, average_precision_score, classification_report
from sklearn.model_selection import train_test_split, RandomizedSearchCV, RepeatedStratifiedKFold, StratifiedKFold

from metaorf.modeling.etl import generate_orf_id, load_features, load_truth_datasets
from metaorf.modeling.ensemble import Dataset, plot_roc_pr, plot_roc, plot_pr

from pathlib import Path

pd.set_option('display.max_columns', 100)

warnings.filterwarnings('ignore')

In [2]:
data_dir = Path('../data').absolute()

def download_feature_set(experiment_name, local_path):
    orf_path = f"s3://velia-piperuns-dev/{experiment_name}/output/{experiment_name}_orf_features.csv"
    local_path = f"{local_path}/{experiment_name}_orf_features.csv"
    cmd = f"aws s3 cp {orf_path} {local_path}"
    print(cmd)
    os.system(cmd)
    return local_path

for experiment_name in open(f"{data_dir}/experiments.txt"):
    experiment_name = experiment_name.strip().strip("/")
    download_feature_set(experiment_name, data_dir)

### Load truth datasets

In [3]:
#truth_df = shuffle(pd.read_csv("s3://velia-piperuns-dev/summary/truthset_batch1to4_240411_2.txt", sep='\t'))
#truth_df['orf_id'] = truth_df.apply(lambda x: generate_orf_id(x), axis=1)

In [4]:
#batch3 = [name for name in set(truth_df["ORF.type"]) if (name.startswith("batch3"))]
#truth_df = truth_df[~ truth_df["ORF.type"].isin(batch3)]

In [3]:
truth_pos_df = pd.read_csv(data_dir.joinpath('truthset_positives.240227.tsv'), sep='\t')
truth_neg_df = pd.read_csv(data_dir.joinpath('truthset_negatives.240227.tsv'), sep='\t')
truth_pos_df['orf_id'] = truth_pos_df.apply(lambda x: generate_orf_id(x), axis=1)
truth_neg_df['orf_id'] = truth_neg_df.apply(lambda x: generate_orf_id(x), axis=1)
truth_df = shuffle(pd.concat([truth_pos_df, truth_neg_df]))

In [4]:
datasets = load_truth_datasets(truth_df, data_dir, overwrite=True, dataset_names=['iPSC', 'MB1', 'Gaertner'])

In [5]:
dataset_name = 'all'
ds = copy.deepcopy(datasets[dataset_name])

In [6]:
sum(ds.y), len(ds.y)

(8836, 118647)

In [7]:
#ds.X = ds.X.drop(columns=["bigprot_id", "size_peak_frac", "size_gini", "periodicity_score",
#                         "pos_1_vs_0", "pos_2_vs_0", "frames_1_vs_0", "frames_2_vs_0"])
ds.X = ds.X.drop(columns=["bigprot_id"])
ds.X

,chrom_id,mean,sum,std,n_reads_orf_vs_genome,pos_1_vs_0,pos_2_vs_0,frames_1_vs_0,frames_2_vs_0,periodicity_first_60_1_vs_0,periodicity_first_60_2_vs_0,periodicity_last_60_1_vs_0,periodicity_last_60_2_vs_0,n_empty_codons,longest_empty_length_whole,longest_empty_length_first_30,longest_empty_length_last_30,five_utr_vs_cds_mean,five_utr_vs_cds_max,five_utr_vs_start_codon_mean,five_utr_vs_start_codon_max,cds_utr_vs_start_codon_mean,cds_utr_vs_start_codon_max,three_utr_vs_cds_mean,three_utr_vs_cds_max,three_utr_vs_stop_codon_mean,three_utr_vs_stop_codon_max,cds_utr_vs_stop_codon_mean,cds_utr_vs_stop_codon_max,dist_neg_100,dist_neg_150,dist_pos_100,dist_pos_150,five_utr_periodicity,five_utr_in_frame_bins,three_utr_periodicity,three_utr_in_frame_bins,price,ribotish,ribocode,tis_transformer_score,size_peak_frac,size_gini,periodicity_score
0,chr1,1.799196,-0.516774,0.035301,1.000000,0.00000,0.000000,0.101266,0.379747,0.000000,0.050000,0.035714,0.607143,-1.505150,-1.414973,-1.230449,-1.079181,0.000000,0.698970,1.079181,0.698970,3.802112e-01,0.221849,1.179128,0.845098,1.544068,0.698970,0.397940,-0.146128,-1.255273,-1.778151,-0.477121,-0.477121,0.000000,0.0,0.0,0.1,7.638348,8.148810,7.012288,0.002689,0.688450,0.683772,0.683194
1,chr1,1.280714,-1.296777,0.097380,0.986014,0.00000,0.181818,0.150407,0.282520,0.115044,0.247788,0.208955,0.358209,-1.176091,-1.146128,-0.778151,-0.903090,1.038491,1.000000,1.085629,0.564271,5.551732e-02,-0.435729,2.064655,1.146128,1.799341,0.301030,-0.276206,-0.845098,-1.778151,-1.778151,-1.079181,-1.079181,1.800000,0.1,0.0,0.0,999.000000,29.838859,24.775366,0.651046,0.688450,0.683772,0.683194
2,chr1,1.806684,-0.946899,0.035862,0.972222,1.00000,1.000000,0.323699,0.497110,0.186047,0.279070,0.800000,1.050000,-1.919078,-2.053078,-1.176091,-1.204120,1.762714,0.845098,0.000000,-0.000000,0.000000e+00,-0.845098,0.872769,0.602060,1.419129,0.221849,0.611820,-0.380211,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.0,0.0,0.0,999.000000,6.157853,0.000000,0.001963,0.688450,0.683772,0.683194
3,chr1,1.022905,-1.661042,0.165785,1.000000,0.00000,0.000000,0.175318,0.422135,0.211538,0.506410,0.131868,0.357143,-0.845098,-1.176091,-0.845098,-0.698970,0.528212,0.685397,0.046344,-0.511883,-5.278611e-01,-1.197281,1.548616,0.716003,1.780857,0.447158,0.249644,-0.268845,-0.477121,-1.000000,-0.845098,-0.845098,2.000000,0.2,0.0,0.1,999.000000,31.557925,0.000000,0.000013,0.688450,0.683772,0.683194
4,chr1,1.348968,-1.203700,0.102018,1.000000,0.20000,0.400000,0.160112,0.438202,0.100000,0.328571,0.171717,0.464646,-1.462398,-1.531479,-0.954243,-0.698970,3.000000,3.000000,3.000000,3.000000,-3.820633e-01,-0.963788,1.030877,0.330993,0.000000,-0.845098,0.000000,-1.176091,-1.778151,-1.778151,-1.079181,-1.079181,0.000000,0.0,2.0,0.0,999.000000,0.000000,15.126927,0.886215,0.688450,0.683772,0.683194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118642,chr5,1.559996,-1.078494,0.059143,0.974619,10.00000,1.000000,0.087227,0.109034,0.113636,0.170455,0.111111,0.022222,-1.505150,-1.462398,-1.000000,-1.146128,0.972046,0.636822,0.000000,-0.477121,0.000000e+00,-1.113943,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000,0.2,0.0,0.0,0.000000,0.000000,31.169173,0.546552,0.649214,0.808236,0.747891
118643,chr15,1.641093,-0.687287,0.043169,1.000000,0.00000,0.000000,0.104000,0.144000,0.218750,0.062500,0.081081,0.162162,-1.146128,-1.397940,-1.361728,-1.301030,0.500602,0.096910,0.500602,-0.000000,7.201942e-09,-0.096910,3.000000,3.000000,0.000000,0.000000,0.000000,0.000000,-1.079181,-1.778151,-1.431364,-1.778151,0.300000,0.3,0.0,0.0,0.000000,0.000000,14.697003,0.000176,0.649214,0.808236,0.747891
118644,chr15,1.553679,-1.090759,0.064475,0.997475,0.00000,1.000000,0.108696,0.118012,0.125000,0.089286,0.081081,0.162162,-1.518514,-1.505150,-0.778151,-1.301030,0.521871,0.433656,0.488703,0.35902

### Choose model

#### Gradient boosting classifier (standard)

In [9]:
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 8, 10],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 0.1, 0.2, 0.5],
    'subsample': [0.4, 0.6, 0.8, 1.0],
    'loss': ['deviance', 'exponential'],
}

def StratifiedByChrom(X):
    groups = [['chr1', 'chr6', 'chr11', 'chr16'],
            ['chr2', 'chr7', 'chr12', 'chr17'],
            ['chr3', 'chr8', 'chr13', 'chr18', 'chrX'],
            ['chr4', 'chr9', 'chr14', 'chr19'],
            ['chr5', 'chr10', 'chr15', 'chr20', 'chr21', 'chr22', 'chrY']]

    for group_idx in range(len(groups)):
        train_indice = X[~X["chrom_id"].isin(groups[group_idx])].index
        test_indice = X[X["chrom_id"].isin(groups[group_idx])].index
        yield (train_indice.values, test_indice.values)

cv = StratifiedByChrom(ds.X)
gclf = RandomizedSearchCV(GradientBoostingClassifier(), param_grid, random_state=0, n_jobs=72, n_iter=100, cv=cv)
search = gclf.fit(ds.X.drop(columns=["chrom_id"]).values, ds.y)
search.best_params_
classifier = GradientBoostingClassifier(**search.best_params_)

In [10]:
search.best_params_

{'subsample': 1.0,
 'n_estimators': 200,
 'min_samples_split': 4,
 'min_samples_leaf': 2,
 'max_features': 0.1,
 'max_depth': 5,
 'loss': 'exponential',
 'learning_rate': 0.2}

best_params = {'subsample': 0.9,
 'n_estimators': 100,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 0.5,
 'max_depth': 10,
 'loss': 'exponential',
 'learning_rate': 0.05}

In [11]:
best_params = {'subsample': 1.0,
 'n_estimators': 200,
 'min_samples_split': 4,
 'min_samples_leaf': 2,
 'max_features': 0.1,
 'max_depth': 5,
 'loss': 'exponential',
 'learning_rate': 0.2}

In [12]:
classifier = GradientBoostingClassifier(**best_params)
classifier_type = 'gb'

In [13]:
ds.model = classifier

In [14]:
with open(f'../data/top_model_{dataset_name}_gb.pkl', 'wb') as file:
    pickle.dump(ds, file)

In [15]:
feature_importances, roc_pr_fig = plot_roc(ds, classifier)
roc_pr_fig.savefig(f'../figures/roc_curve_{ds.name}_{classifier_type}.png', dpi=200, bbox_inches='tight')

In [16]:
feature_importances, pr_fig = plot_pr(ds.X, ds.y, classifier, n_splits=5)
pr_fig.savefig(f'../figures/pr_curve_{ds.name}_{classifier_type}.png', dpi=200, bbox_inches='tight')

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2)
cloned_classifier = clone(classifier)
fig, [ax_roc, ax_pr] = plt.subplots(1, 2, figsize=(11, 5))

for i, (train, test) in enumerate(cv.split(ds.X, ds.y)):
    cloned_classifier.fit(ds.X.iloc[train], ds.y[train])
    fpr, tpr, roc_thresh = roc_curve(ds.y[test], cloned_classifier.predict_proba(ds.X.iloc[test])[:, 1])
    pr, rc, pr_thresh = precision_recall_curve(ds.y[test], cloned_classifier.predict_proba(ds.X.iloc[test])[:, 1])
    
    ax_roc.plot(fpr, tpr)
    ax_roc.plot(fpr, roc_thresh, color='black')

    ax_pr.plot(rc, pr)
    ax_pr.plot(rc, np.append(pr_thresh, 1), color='black')

fig.savefig(f'../figures/roc_pr_threshold_{dataset_name}_gb.png', dpi=200, bbox_inches='tight')

In [17]:
fi_df = pd.DataFrame(feature_importances, columns=ds.X.drop(columns=['chrom_id']).columns)
medians = fi_df.median()
sorted_columns = medians.sort_values(ascending=False).index
sorted_df = fi_df[sorted_columns]
plot_df = fi_df.melt(var_name='Category', value_name='Value')

In [18]:
bplot = sns.boxplot(plot_df, y='Category', x='Value', order=sorted_df.columns)
bplot.figure.set_size_inches(5,6)
bplot.figure.savefig(f'../figures/feature_importance_impurity_{dataset_name}_{classifier_type}.png', dpi=200, bbox_inches='tight')

In [19]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
corr = spearmanr(ds.X).correlation
corr = (corr + corr.T) / 2
np.fill_diagonal(corr, 1)
distance_matrix = 1 - np.abs(corr)
dist_linkage = hierarchy.ward(squareform(distance_matrix))
dendro = hierarchy.dendrogram(
    dist_linkage, labels=ds.X.columns.to_list(), ax=ax1, leaf_rotation=90
)
dendro_idx = np.arange(0, len(dendro["ivl"]))

ax2.imshow(corr[dendro["leaves"], :][:, dendro["leaves"]])
ax2.set_xticks(dendro_idx)
ax2.set_yticks(dendro_idx)
ax2.set_xticklabels(dendro["ivl"], rotation="vertical")
ax2.set_yticklabels(dendro["ivl"])
_ = fig.tight_layout()

In [ ]:
def plot_permutation_importance(clf, X, y, ax):
    result = permutation_importance(clf, X, y, n_repeats=100, random_state=42, n_jobs=2)
    perm_sorted_idx = result.importances_mean.argsort()

    ax.boxplot(
        result.importances[perm_sorted_idx].T,
        vert=False,
        labels=X.columns[perm_sorted_idx],
    )
    
    ax.axvline(x=0, color="k", linestyle="--")
    return ax

In [ ]:
cluster_ids = hierarchy.fcluster(dist_linkage, 1, criterion="distance")
cluster_id_to_feature_ids = defaultdict(list)
for idx, cluster_id in enumerate(cluster_ids):
    cluster_id_to_feature_ids[cluster_id].append(idx)
selected_features = [v[0] for v in cluster_id_to_feature_ids.values()]
selected_features_names = ds.X.columns[selected_features]

cloned_classifier = clone(classifier)
cloned_classifier.fit(ds.X[selected_features_names], ds.y)

fig, ax = plt.subplots(figsize=(7, 6))
plot_permutation_importance(cloned_classifier, ds.X[selected_features_names], ds.y, ax)
ax.set_title("Permutation Importances on selected subset of features\n(test set)")
ax.set_xlabel("Decrease in accuracy score")
ax.figure.tight_layout()
plt.show()
    